In [1]:
import cv2
import time
import pandas as pd
from ultralytics import YOLO
from datetime import datetime

# -----------------------------
# 🔧 SETUP
# -----------------------------
model = YOLO("yolov8n.pt")
cap = cv2.VideoCapture(0)  # or replace with video path

if not cap.isOpened():
    print("❌ Camera not detected.")
    exit()

# Get video resolution
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS)) or 30

# Define video output filename
output_filename = f"pet_activity_recorded_{datetime.now().strftime('%Y%m%d_%H%M%S')}.mp4"

# Define video codec & output file
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_filename, fourcc, fps, (frame_width, frame_height))

print(f"✅ Pet activity monitoring & recording started. Output file: {output_filename}")
print("🔴 Press 'q' to stop recording.")

# -----------------------------
# 🧠 VARIABLES
# -----------------------------
prev_area = 0
last_activity = "None"
activity_log = []

# -----------------------------
# 🚀 MAIN LOOP
# -----------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO detection
    results = model(frame)
    boxes = results[0].boxes
    labels = []
    total_area = 0

    for box in boxes:
        cls = int(box.cls[0])
        label = model.names[cls]
        labels.append(label)

        x1, y1, x2, y2 = map(int, box.xyxy[0])
        total_area += (x2 - x1) * (y2 - y1)

        # Draw detection box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

    # Activity estimation based on total movement area
    movement = abs(total_area - prev_area)
    prev_area = total_area
    activity = "No Pet Detected"

    if any(pet in labels for pet in ["dog", "cat"]):
        if movement > 5000:
            activity = "Playing"
        elif "bowl" in labels:
            activity = "Eating"
        else:
            activity = "Sleeping / Idle"

    # Display on frame
    cv2.putText(frame, f"Activity: {activity}",
                (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 3)

    # Log when activity changes
    if activity != last_activity:
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_entry = {"time": timestamp, "activity": activity}
        activity_log.append(log_entry)
        print(f"[{timestamp}] Activity changed → {activity}")
        last_activity = activity

    # Write the frame to video
    out.write(frame)

    # Display live video feed
    cv2.imshow("🐾 Pet Activity Monitor + Recording", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("🛑 Stopping...")
        break

# -----------------------------
# 💾 SAVE LOGS AND CLEANUP
# -----------------------------
if activity_log:
    df = pd.DataFrame(activity_log)
    df.to_csv("pet_activity_log.csv", index=False)
    print("✅ Activity log saved to pet_activity_log.csv")

cap.release()
out.release()
cv2.destroyAllWindows()

print(f"🎞️ Recorded video saved as: {output_filename}")


✅ Pet activity monitoring & recording started. Output file: pet_activity_recorded_20251211_173226.mp4
🔴 Press 'q' to stop recording.

0: 480x640 1 person, 1 cat, 421.8ms
Speed: 11.3ms preprocess, 421.8ms inference, 32.7ms postprocess per image at shape (1, 3, 480, 640)
[2025-12-11 17:32:28] Activity changed → Playing

0: 480x640 1 person, 1 tv, 1 laptop, 266.8ms
Speed: 5.0ms preprocess, 266.8ms inference, 3.4ms postprocess per image at shape (1, 3, 480, 640)
[2025-12-11 17:32:28] Activity changed → No Pet Detected

0: 480x640 1 person, 1 tv, 1 laptop, 222.9ms
Speed: 2.9ms preprocess, 222.9ms inference, 3.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 tv, 1 laptop, 208.1ms
Speed: 3.1ms preprocess, 208.1ms inference, 4.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 223.7ms
Speed: 3.4ms preprocess, 223.7ms inference, 5.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 laptop, 224.2ms
Speed: 4.0ms preprocess,